In [21]:
from os.path import join
import sys
import pandas as pd
import geopandas as gpd

sys.path.append(r'c:\Users\dmarin\Dropbox\Pro\info\Git\Project_repositories\python_odc_libs')
import analytics_odc_lib as agf

path_file = r'c:\Users\dmarin\Dropbox\Pro\OpenDC\99 Projects\0142-GD4H - Datactivist\00 Incoming\230717 Fichier multiples coordonnees - From Mamadou\tab_SpF.zip'
dir_export = r'c:\Users\dmarin\Dropbox\Pro\OpenDC\99 Projects\0142-GD4H - Datactivist\02 Work\01 Tables\csv'
print('ok')

Numpy future warnings deactivated...
Could not import topojson
Could not import simpledbf (installation with pip, not conda)
Could not import scipy
Could not import pandasql
ok


In [5]:
df = pd.read_csv(path_file, sep=';', encoding='ansi', low_memory=False)
print('{:,} records'.format(len(df.index)))
df.head(1)

1,211,642 records


,milieu,annee,code_etab,nom_etablissement,adresse,commune,code_insee,Type de rejet,Polluant,CAS,...,masse_accidentelle,longitude,latitude,Type de coordonnées,Type de projection,annee_la_plus_recente,longitude_annee_recente,latitude_annee_recente,Type de coordonnées_annee_recente,Type de projection_annee_recente
0,Eau,2002,0005100143,CIE GREENFIELD,ZI de la GRANDE BORNE,CHATEAU-THIERRY,02168,Rejet Isolé,Carbone organique total (COT),NaN,...,NaN,3.39240,49.03226,Vérifié,WGS84,2022,3.3924,49.03226,Vérifié,WGS84


In [8]:
col_check = ['Type de coordonnées', 'Type de projection', 'Type de coordonnées_annee_recente', 'Type de projection_annee_recente']
for col in col_check:
    df_group = df.groupby(col, as_index=False)['annee'].count()
    sum_val = df_group['annee'].sum()
    df_group['%'] = 100 * df_group['annee'] / sum_val
    df_group.drop(columns='annee', inplace=True)
    print('--------------------\n{}:\n{}'.format(col, df_group))

--------------------
Type de coordonnées:
  Type de coordonnées          %
0           Centroide  13.250283
1             Déclaré  33.774745
2             Vérifié  52.974971
--------------------
Type de projection:
  Type de projection          %
0         Lambert II  29.506582
1              WGS84  70.493418
--------------------
Type de coordonnées_annee_recente:
  Type de coordonnées_annee_recente          %
0                         Centroide   4.540533
1                           Déclaré  42.484496
2                           Vérifié  52.974971
--------------------
Type de projection_annee_recente:
  Type de projection_annee_recente          %
0                       Lambert II  37.804463
1                            WGS84  62.195537


In [12]:
# # Export lambert II and WGS 84 separately:
f_types = ['Type de projection', 'Type de projection_annee_recente']
f_syst = ['Lambert II', 'WGS84']
for f_type in f_types:
    for syst in f_syst:
        f_export = join(dir_export, '{}_{}.csv'.format(f_type, syst).replace(' ', ''))
        df_exp = df.loc[df[f_type]==syst]
        print('Exporting {:,} records:\t{}\t{}'.format(len(df_exp.index), f_type, syst))
        df_exp.to_csv(f_export, index=False, encoding='ansi')

print('ok')


Exporting 353,570 records:	Type de projection	Lambert II
Exporting 844,705 records:	Type de projection	WGS84
Exporting 457,744 records:	Type de projection_annee_recente	Lambert II
Exporting 753,076 records:	Type de projection_annee_recente	WGS84
ok


In [19]:
# # Test automatic correction:
wgs84_bounds = {
    'minx': -180,
    'maxx': 180,
    'miny': -180,
    'maxy': 180,
}
lambert_II_bounds = {}
epsg_lambert_II = 27572
epsg_wgs84 = 4326

# # Laod region lambert II and get bounds:
reg_l_II = gpd.read_file(r'c:\Users\dmarin\Dropbox\Pro\OpenDC\99 Projects\0142-GD4H - Datactivist\02 Work\00 GIS\00 SHP\admin\regions_convex_hull_Lambert_II.shp')
bbox = reg_l_II.geometry.bounds
lambert_II_bounds['minx'] = bbox.iloc[0, 0]
lambert_II_bounds['miny'] = bbox.iloc[0, 1]
lambert_II_bounds['maxx'] = bbox.iloc[0, 2]
lambert_II_bounds['maxy'] = bbox.iloc[0, 3]

print(lambert_II_bounds)
bbox


{'minx': 72525.72071506886, 'miny': 1614430.463979396, 'maxx': 1223160.3603614653, 'maxy': 2683676.9260735745}


,minx,miny,maxx,maxy
0,72525.720715,1.614430e+06,1.223160e+06,2.683677e+06


In [38]:
# # Base field names:
f_lng_base = 'longitude'  # base name, can be lng or Lambert II
f_lat_base = 'latitude'
f_lng_rec_base = 'longitude_annee_recente'
f_lat_rec_base = 'latitude_annee_recente'

# # Corrected field names:
f_lng = '{} WGS84'.format(f_lng_base)
f_lat = '{} WGS84'.format(f_lat_base)
f_lng_rec = '{} WGS84'.format(f_lng_rec_base)
f_lat_rec = '{} WGS84'.format(f_lat_rec_base)


def coord_to_wgs84(df, col_coord=[], col_wgs84=[]):
    """
    Function that transforms the columns in col_coord that are not in the WGS84 bounds into Lambert II, convert to WGS84 and create consistent WGS84 columns
    :param df:
    :param col_coord:
    :param col_wgs84:
    :return:
    """
    df2 = df.reset_index(drop=True)
    # # Tmp field names:
    f_key = 'key'  # for merge
    df2.loc[:, f_key] = df2.index.values
    f_x = 'x_coord'
    f_y = 'y_coord'

    # # Convert coordinates to float:
    df2.loc[:, f_x] = pd.to_numeric(df[col_coord[0]], errors='coerce')
    df2.loc[:, f_y] = pd.to_numeric(df[col_coord[1]], errors='coerce')

    # # Extract records for which coordinates are in Lambert II
    idx_lII =df2.loc[(df2[f_x]>wgs84_bounds['maxx']) & (df2[f_y]>wgs84_bounds['maxy'])].index
    df_lII = df2.loc[idx_lII, [f_key, f_x, f_y]]

    # # Create geodf with epsg Lambert II and project to lat, lng:
    gdf_lII = agf.df_to_gdf(df=df_lII, col_geom=[f_x, f_y], epsg=epsg_lambert_II)
    gdf_wgs84 = gdf_lII.to_crs(epsg=epsg_wgs84)
    gdf_wgs84.loc[:, f_x] = gdf_wgs84.geometry.x
    gdf_wgs84.loc[:, f_y] = gdf_wgs84.geometry.y

    # # Merge with df2:
    df2 = df2.merge(gdf_wgs84[[f_key, f_x, f_y]], on=f_key, how='left')

    # # Replace col WGS 84:
    idx_na = df2.loc[df2['{}_y'.format(f_x)].isna()].index
    idx_non_na = df2.loc[~df2['{}_y'.format(f_x)].isna()].index
    # # Replace by original where already WGS84:
    df2.loc[idx_na, col_wgs84[0]] = df2.loc[idx_na, col_coord[0]]
    df2.loc[idx_na, col_wgs84[1]] = df2.loc[idx_na, col_coord[1]]
    # # Replace by converted where was Lambert II:
    df2.loc[idx_non_na, col_wgs84[0]] = df2.loc[idx_non_na, '{}_y'.format(f_x)]
    df2.loc[idx_non_na, col_wgs84[1]] = df2.loc[idx_non_na, '{}_y'.format(f_y)]

    # # Convert to numeric:
    df2[col_wgs84[0]] = pd.to_numeric(df2[col_wgs84[0]], errors='coerce')
    df2[col_wgs84[1]] = pd.to_numeric(df2[col_wgs84[1]], errors='coerce')

    # # Drop tmp columns:
    df2.drop(columns=[f_key, '{}_x'.format(f_x), '{}_x'.format(f_y), '{}_y'.format(f_x), '{}_y'.format(f_y)], inplace=True)

    return df2

# # Longitude latiude:
df_fixed = coord_to_wgs84(df=df, col_coord=[f_lng_base, f_lat_base], col_wgs84=[f_lng, f_lat])

# # Longitude latiude année récente:
df_fixed = coord_to_wgs84(df=df_fixed, col_coord=[f_lng_rec_base, f_lat_rec_base], col_wgs84=[f_lng_rec, f_lat_rec])

print('{:,} records (vs {:,} original)'.format(len(df_fixed.index), len(df.index)))

df_fixed.to_csv(join(dir_export, 'projection_harmonisee.csv'), index=False)
df_fixed.sample(10)



1,211,642 records (vs 1,211,642 original)


,milieu,annee,code_etab,nom_etablissement,adresse,commune,code_insee,Type de rejet,Polluant,CAS,...,Type de projection,annee_la_plus_recente,longitude_annee_recente,latitude_annee_recente,Type de coordonnées_annee_recente,Type de projection_annee_recente,longitude WGS84,latitude WGS84,longitude_annee_recente WGS84,latitude_annee_recente WGS84
80052,Eau,2006,0005401338,Rhodia OPERATIONS Uisne de Clamecy,"CI Saint Roch, BP 25",CLAMECY,58079,Rejet Isolé,Demande chimique en oxygène (DCO),NaN,...,WGS84,2022,3.52114,4.746527e+01,Vérifié,WGS84,3.521140,47.465270,3.521140,47.465270
891776,Air,2013,059.01186,NOUVION,Le Village,LOEUILLEY,70305,NaN,Composés organiques volatils non méthaniques (...,NaN,...,WGS84,2015,5.44431,4.740480e+01,Déclaré,WGS84,5.444310,47.404800,5.444310,47.404800
76633,Eau,2005,24370002,FROMAGERIE DES CHAUMES,LES LAURENTS,SAINT-ANTOINE-DE-BREUILH,24370,Rejet Isolé,Matières en suspension (MES),NaN,...,Lambert II,2016,428647.00000,1.983803e+06,Déclaré,Lambert II,0.170309,44.833712,0.170309,44.833712
651893,Air,2004,070.01004,INOVA (CENTRE DE TRAITEMENT THERMIQUE DE NOYEL...,RUE DU DOCTEUR SCHAFFNER,NOYELLES-SOUS-LENS,62628,NaN,Poussières totales (TSP),NaN,...,Lambert II,2018,2.85000,5.042000e+01,Déclaré,Lambert II,2.850000,50.420000,2.850000,50.420000
747390,Air,2008,064.01784,Thales Alenia Space CANNES,100 Bd du Midi-B.P. 99,CANNES,06029,NaN,Nickel et ses composés (Ni),7440-02-0,...,Lambert II,2018,973160.00000,1.484000e+05,Déclaré,Lambert II,5.976826,28.526143,5.977058,28.528711
205020,Eau,2009,063.01005,FAMAT,ZI de BRAIS BP 218,SAINT-NAZAIRE,44184,Rejet raccordé,Aluminium et ses composés (Al),7429-90-5,...,WGS84,2018,250121.00000,2.265064e+06,Déclaré,Lambert II,-2.250000,47.280000,-2.292101,47.292533
800421,Air,2010,061.02152,Carrier Montluel,BP 49 Route de Thil,MONTLUEL,01262,NaN,Hydroflurocarbures (HFC),NaN,...,Lambert II,2018,810500.00000,2.097573e+06,Déclaré,Lambert II,5.046712,45.841534,5.046921,45.845787
684546,Air,2006,058.00360,Société Couronnaise de Raffinage,72 rue Aristide Briand,PETIT-COURONNE,76497,NaN,Méthane (CH4),74-82-8,...,Lambert II,2012,503.70600,2.487026e+03,Déclaré,Lambert II,-3.401492,27.162504,-3.401492,27.162504
44843,Eau,2004,065.01845,Routière de l'Est Parisien,"RD 215, Lieudit 'les Bonnes'",MOISENAY,77295,Rejet Isolé,Cadmium et ses composés (Cd),7440-43-9,...,Lambert II,2018,631221.00000,2.396817e+06,Déclaré,Lambert II,2.759403,48.569307,2.759403,48.569307
913605,Air,2013,75600322,GAEC JAIGU PELLOUOIS,Les 13 Chênes,TREMOREL,22371,NaN,Ammoniac (NH3),7664-41-7,...,WGS84,2016,-2.30000,4.819000e+01,Centroide,WGS84,-2.300000,48.190000,-2.300000,48.190000


In [46]:
# # Check number of records in bounds:
reg_l_II_wgs84 = reg_l_II.to_crs(epsg=epsg_wgs84)

def check_nb_in_france(df, col_geom=[f_lng, f_lat], comment=''):
    """
    Tests how many fixed records are in France
    :param df:
    :param col_geom:
    :return:
    """
    # # Convert to numeric:
    df2 = df.copy()
    df2[col_geom[0]] = pd.to_numeric(df2[col_geom[0]], errors='coerce')
    df2[col_geom[1]] = pd.to_numeric(df2[col_geom[1]], errors='coerce')

    gdf_tmp = agf.df_to_gdf(df=df2[col_geom], col_geom=col_geom, epsg=epsg_wgs84)
    df_fixed_check = agf.spatial_join(gdf_left=gdf_tmp, gdf_right=reg_l_II_wgs84, op='within', how='left').reset_index(drop=True)
    idx_in = df_fixed_check.loc[~df_fixed_check['region'].isna()].index
    idx_out = df_fixed_check.loc[df_fixed_check['region'].isna()].index
    nb_in = len(idx_in)
    nb_out = len(idx_out)
    print('{}: {:,} en France ({:.1%})'.format(comment, nb_in, nb_in/(nb_in+nb_out)))
    # # Export outliers:
    df.loc[idx_out].to_csv(join(dir_export, 'coord_hors_france_{}.csv'.format(comment)), index=False, encoding='ansi')

    return df_fixed_check

# # Test lat, lng avant harmonisation:
check_nb_in_france(df=df, col_geom=[f_lng_base, f_lat_base], comment='col_long_lat_raw_data')

# # Test lat, lng après harmonisation:
check_nb_in_france(df=df_fixed, col_geom=[f_lng, f_lat], comment='col_long_lat')

# # Test recent après harmonisation:
check_nb_in_france(df=df_fixed, col_geom=[f_lng_rec, f_lat_rec], comment='col_long_lat_annee_recente')

col_long_lat_raw_data: 846,458 en France (69.9%)
col_long_lat: 1,081,595 en France (89.3%)
col_long_lat_annee_recente: 1,072,422 en France (88.5%)


,longitude_annee_recente WGS84,latitude_annee_recente WGS84,geometry,region,new_code,area,perimeter
0,3.39240,49.03226,POINT (3.39240 49.03226),Hauts-de-France,32,8.228557e+11,3.500891e+06
1,3.39240,49.03226,POINT (3.39240 49.03226),Hauts-de-France,32,8.228557e+11,3.500891e+06
2,3.22590,49.60741,POINT (3.22590 49.60741),Hauts-de-France,32,8.228557e+11,3.500891e+06
3,3.22590,49.60741,POINT (3.22590 49.60741),Hauts-de-France,32,8.228557e+11,3.500891e+06
4,3.43197,49.94787,POINT (3.43197 49.94787),Hauts-de-France,32,8.228557e+11,3.500891e+06
...,...,...,...,...,...,...,...
1211637,1.18239,49.31161,POINT (1.18239 49.31161),Hauts-de-France,32,8.228557e+11,3.500891e+06
1211638,1.18239,49.31161,POINT (1.18239 49.31161),Hauts-de-France,32,8.228557e+11,3.500891e+06
1211639,1.18239,49.31161,POINT (1.18239 49.31161),Hauts-de-France,32,8.228557e+11,3.500891e+06
1211640,1.18239,49.31161,POINT (1.18239 49.31161),Hauts-de-France,32,8.228557e+11,3.500891e+06
